In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Local:**

In [ ]:
import constants
import constraints

from datagolf import DataGolf
from fileman import fileman
from pga import PGA
from picklejar import PickleJar

from dataprep import DataPrep
from optimizer import Optimizer

In [ ]:
Optimizer.Run()